# Analyse des temps de paroles des élections europeenes

Preparations des fichiers pour affichage.<br>
<br>
Le site de l'arcom nous fournie plusieurs fichiers à l'adresse suivante : <br>
<a href = "https://www.arcom.fr/actualites/elections-legislatives-2024-publication-des-releves-des-temps-de-parole"> page arcom </a><br>


In [1]:
# import des bibliotheques
import pandas as pd
import os


In [2]:
#lister les nom des fichiers dans les données brutes
liste_fichier =os.listdir('donnee_arcom_leg/1tour')


In [3]:
# faire les trucs pour tout les fichiers :
temps_parole_leg2024 = pd.DataFrame()
for element in liste_fichier:
    # lire le fichier 
    tempo_df = pd.read_csv('donnee_arcom_leg/1tour/'+element,encoding='ISO-8859-3',sep=';',header=3)
    # garder les 2 derniere ligne du temps presidentiel
    df_president = tempo_df.tail(2)
    # supprimer les ligne du temps presidentiel
    tempo_df = tempo_df[:-2]
    # rajouter une ligne pour le temps presidentiel 
    tempo_df.loc[len(tempo_df)]=['temps_president',df_president.iat[1,1]]
    # rajouter le nom de la chaine
    tempo_df['Nom du media'] = (element[31:-4])
    tempo_df = tempo_df[tempo_df['Partis politiques']!='Total des temps de parole']
    # concat les dataframe entre eux
    temps_parole_leg2024 = pd.concat([temps_parole_leg2024,tempo_df])
    

In [4]:
#couleur_pol = pd.read_csv('donnee_contruite/couleurs.csv',encoding='ISO-8859-3')

Liste des medias contenante les fichiers :<br>
Canal plus n'a pas fourni de temps de paroles<br>

In [5]:
print(temps_parole_leg2024['Nom du media'].unique())

['France3' 'C8' 'TV5-Monde' 'RadioClassique' 'M6' 'RFI' 'RMC' 'France2'
 'France5' 'RMC-Story' 'FranceInter' 'RTL' 'TMC' 'FranceCulture' 'Cnews'
 'Europe1' 'TF1' 'LCI' 'franceinfo_' 'FranceInfo' 'BFMTV' 'France24'
 'SudRadio']


In [6]:
temps_parole_leg2024['Partis politiques'].unique()

array(['La France insoumise', 'Les Ecologistes', 'Les Républicains',
       'Parti communiste français', 'Parti socialiste', 'temps_president',
       'Place publique', 'Rassemblement national', 'Reconquête',
       'Renaissance', 'LES REPUBLICAINS', 'PARTI COMMUNISTE',
       'PARTI SOCIALISTE', 'Divers Gauche', 'Horizons',
       'Mouvement démocrate', 'Divers Centre', 'Divers Droite',
       'Territoires de progrès', 'Debout la France',
       'Union des démocrates et indépendants', 'EQUINOXE', 'RESISTONS',
       'Génération.s', 'Fédération progressiste', 'Agir',
       'Les Centristes', 'Nouveau parti anticapitaliste'], dtype=object)

In [7]:
# listez les media en fonction de leur categories
liste_tv_gen = ['TMC', 'France2', 'France3','M6', 'RMC-Story',
       'TF1', 'France5', 'C8', 'RMCDecouverte', 'TV5-Monde',
       ]

liste_tv_info = ['LCI', 'franceinfo_',
       'BFMTV', 'France24', 'Euronews', 'Cnews']

liste_radio = ['RTL', 'RFI', 'FranceInfo', 'FranceCulture',
       'SudRadio', 'RMC', 'Europe1', 'BFMBusiness',
       'RadioClassique', 'FranceInter']

# ajouter les categories dans le dataframe :
temps_parole_leg2024['Type de media'] = 'inconnu'

for media in liste_tv_gen: 
       temps_parole_leg2024.loc[temps_parole_leg2024['Nom du media']==media,'Type de media'] = 'TV généraliste'
for media in liste_tv_info:
       temps_parole_leg2024.loc[temps_parole_leg2024['Nom du media']==media,'Type de media'] = 'TV info continue' 
for media in liste_radio:
       temps_parole_leg2024.loc[temps_parole_leg2024['Nom du media']==media,'Type de media'] = 'radio' 

Catégorisation des médias : <br>

In [8]:
temps_parole_leg2024['Type de media'].unique()

array(['TV généraliste', 'radio', 'TV info continue'], dtype=object)

In [9]:
temps_parole_leg2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287 entries, 0 to 13
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Partis politiques      287 non-null    object
 1   Total période (durée)  287 non-null    object
 2   Nom du media           287 non-null    object
 3   Type de media          287 non-null    object
dtypes: object(4)
memory usage: 11.2+ KB


In [10]:
# voir pour changer le type de données de la periode tout convertir en seconde puis en heure
temps_parole_leg2024['heure'] = temps_parole_leg2024['Total période (durée)'].str[:2]
temps_parole_leg2024['minute'] = temps_parole_leg2024['Total période (durée)'].str[3:5]
temps_parole_leg2024['seconde'] = temps_parole_leg2024['Total période (durée)'].str[-2:]

temps_parole_leg2024['heure'] = temps_parole_leg2024['heure'].astype(int)
temps_parole_leg2024['minute'] = temps_parole_leg2024['minute'].astype(int)
temps_parole_leg2024['seconde'] = temps_parole_leg2024['seconde'].astype(int)

temps_parole_leg2024['Durée en seconde'] = temps_parole_leg2024['heure']*3600 + temps_parole_leg2024['minute']*60 +temps_parole_leg2024['seconde']
temps_parole_leg2024['Durée en minute'] = temps_parole_leg2024['Durée en seconde']/60 

In [11]:
temps_parole_leg2024.drop(columns=['heure','minute','seconde'],inplace=True)

# Temps de parole en secondes des listes :<br>

In [12]:
temps_parole_leg2024.groupby(by='Partis politiques').agg({'Durée en seconde':'sum'}).reset_index()

,Partis politiques,Durée en seconde
0,Agir,367
1,Debout la France,1634
2,Divers Centre,7662
3,Divers Droite,11596
4,Divers Gauche,1754
5,EQUINOXE,6
6,Fédération progressiste,2066
7,Génération.s,1753
8,Horizons,17625
9,LES REPUBLICAINS,168


In [13]:
temps_parole_leg2024.groupby(by='Nom du media').agg({'Durée en seconde':'sum'}).reset_index()

,Nom du media,Durée en seconde
0,BFMTV,145993
1,C8,8243
2,Cnews,53538
3,Europe1,31323
4,France2,11624
5,France24,18281
6,France3,3646
7,France5,14981
8,FranceCulture,4037
9,FranceInfo,55619


In [14]:
# calcul du temps total :

df_temps_total = temps_parole_leg2024.groupby(by='Nom du media').agg({'Durée en seconde':'sum'}).reset_index().rename(columns={'Durée en seconde':'Temps total'})

In [15]:
temps_parole_leg2024 = pd.merge(temps_parole_leg2024,df_temps_total,on='Nom du media',how='inner')
# temps_parole_leg2024 = pd.merge(temps_parole_leg2024,couleur_pol,on='Listes candidates',how='inner')

In [16]:
# calcul des pourcentages des interventions par chaines

temps_parole_leg2024['Taux temps de parole'] = temps_parole_leg2024['Durée en seconde'] / temps_parole_leg2024['Temps total'] *100

# verification opération

temps_parole_leg2024.groupby(by='Nom du media').agg({'Taux temps de parole':'sum'}).reset_index()

,Nom du media,Taux temps de parole
0,BFMTV,100.0
1,C8,100.0
2,Cnews,100.0
3,Europe1,100.0
4,France2,100.0
5,France24,100.0
6,France3,100.0
7,France5,100.0
8,FranceCulture,100.0
9,FranceInfo,100.0


In [17]:
temps_parole_leg2024['Durée en heure'] = temps_parole_leg2024['Durée en seconde']/3600

In [18]:
#bon je change le franceinfo par france info TV pour eviter les confusion entre la TV et la radio
temps_parole_leg2024['Nom du media'] = temps_parole_leg2024['Nom du media'].replace({'franceinfo_':'France info TV',
'SudRadio':'Sudradio',
'TV5-Monde':'TV5Monde'})

In [19]:
# ajout proprio au dataset
media_proprio = pd.read_csv('donnee_contruite/media.csv',encoding='ISO-8859-3')

In [20]:
temps_parole_leg2024 = pd.merge(temps_parole_leg2024,media_proprio,on='Nom du media',how='inner')

In [21]:
temps_parole_leg2024.to_csv('donnee_contruite/temps_paroles_leg2024.csv',index=False)

In [22]:
fichier =pd.read_csv('donnee_contruite/temps_paroles_leg2024.csv')

In [23]:
fichier.head()

,Partis politiques,Total période (durée),Nom du media,Type de media,Durée en seconde,Durée en minute,Temps total,Taux temps de parole,Durée en heure,proprio du media
0,La France insoumise,00:20:22,France3,TV généraliste,1222,20.366667,3646,33.516182,0.339444,France Télévisions
1,Les Ecologistes,00:00:27,France3,TV généraliste,27,0.450000,3646,0.740538,0.007500,France Télévisions
2,Les Républicains,00:00:20,France3,TV généraliste,20,0.333333,3646,0.548546,0.005556,France Télévisions
3,Parti communiste français,00:00:09,France3,TV généraliste,9,0.150000,3646,0.246846,0.002500,France Télévisions
4,Parti socialiste,00:00:16,France3,TV généraliste,16,0.266667,3646,0.438837,0.004444,France Télévisions


In [24]:
fichier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Partis politiques      287 non-null    object 
 1   Total période (durée)  287 non-null    object 
 2   Nom du media           287 non-null    object 
 3   Type de media          287 non-null    object 
 4   Durée en seconde       287 non-null    int64  
 5   Durée en minute        287 non-null    float64
 6   Temps total            287 non-null    int64  
 7   Taux temps de parole   287 non-null    float64
 8   Durée en heure         287 non-null    float64
 9   proprio du media       287 non-null    object 
dtypes: float64(3), int64(2), object(5)
memory usage: 22.6+ KB


In [25]:
fichier.describe()

,Durée en seconde,Durée en minute,Temps total,Taux temps de parole,Durée en heure
count,287.000000,287.000000,287.000000,287.000000,287.000000
mean,2256.700348,37.611672,34588.682927,8.013937,0.626861
std,4399.691735,73.328196,43480.000954,9.924594,1.222137
min,1.000000,0.016667,533.000000,0.024771,0.000278
25%,96.500000,1.608333,6649.000000,1.008517,0.026806
50%,670.000000,11.166667,14981.000000,4.127580,0.186111
75%,1880.500000,31.341667,53538.000000,11.750560,0.522361
max,31506.000000,525.100000,145993.000000,64.509051,8.751667
